In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
with open("../data/Migros_branches/Migros Filialen.html", 'r') as f:
    soup = BeautifulSoup(f.read(), 'html')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 204615: character maps to <undefined>